In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
v_file_date

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
results_schema = 'resultId INT, raceId INT, driverId INT, constructorId INT, number INT, grid INT, position INT, positionText STRING, positionOrder INT, points FLOAT, laps INT, time STRING, milliseconds INT, fastestLap INT, rank INT, fastestLapTime STRING, fastestLapSpeed FLOAT, statusId INT'


results_df = spark.read.schema(results_schema).json(f'{raw_folder_path}/{v_file_date}/results.json')

In [0]:
from pyspark.sql.functions import lit

results_with_column = results_df.withColumnRenamed('resultId','result_id') \
.withColumnRenamed('raceId','race_id') \
.withColumnRenamed('driverId','driver_id') \
.withColumnRenamed('constructorId','constructor_id') \
.withColumnRenamed('positionText','position_text') \
.withColumnRenamed('positionOrder','position_order') \
.withColumnRenamed('fastestLap','fastest_lap') \
.withColumnRenamed('fastestLapTime','fastest_lap_time') \
.withColumnRenamed('fastestLapSpeed','fastest_lap_speed') \
.withColumn("data_source", lit(v_data_source)) \
.withColumn("file_date", lit(v_file_date))\
.drop('statusId')

In [0]:

results_final_df = add_ingestion_date(results_with_column)


De-duped DF

In [0]:
results_deduped_df  = results_final_df.dropDuplicates(['race_id','driver_id'])

In [0]:
# for race_id_list in results_final_df.select('race_id').distinct().collect():
#     if(spark._jsparkSession.catalog().tableExists("f1_processed.results")):
#         spark.sql(f"ALTER TABLE f1_processed.results DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")

In [0]:
# results_final_df.write.mode('append').partitionBy('race_id').format('parquet').saveAsTable('f1_processed.results')

In [0]:
%sql
--Drop table if exists f1_processed.results

In [0]:
# overwrite_partition(results_final_df,'race_id', 'results', 'f1_processed')

In [0]:
merge_condition = "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"
merge_delta_data(results_deduped_df,'f1_processed', 'results', processed_folder_path, merge_condition, 'race_id')

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql
select race_id, driver_id, count(*) as cnt
from f1_processed.results
group by race_id, driver_id
Having cnt > 1
order by race_id, driver_id desc;